In [ ]:
!pip install transformers

In [ ]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 9.4 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [ ]:
!pip install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=daa22ae3f016fe12c4636022ed49339bbd95e7cc9e9067d45133a17eb722d61b
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor


In [ ]:
!pip install ktrain

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from __future__ import division, print_function
from gensim import models
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import nltk
import os
import collections
import re
import string

import preprocessor as p
import ktrain
from ktrain import text
import nltk
from sklearn.preprocessing import LabelEncoder


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [ ]:
df = pd.read_csv("preprocess_data.csv",index_col=0)
df= df.dropna()
df.head()

,Unnamed: 0.1,_id,text,task_1,task_2,text_clean
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN,made amp amp onli abl start make money sustain...
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN,technic still turn back clock dick head
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE,govt stop think world media liber gang ani opt...
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN,would better ask think sleazi shitbag lmao


In [ ]:
LE = LabelEncoder()
LE.fit(df['task_2'])
classes= list(LE.classes_)
classes

['HATE', 'NONE', 'OFFN', 'PRFN']

In [ ]:
df['task_2'] = LE.fit_transform(df['task_2'])
task1_params = dict(zip(LE.classes_, LE.transform(LE.classes_)))
df.head()

,Unnamed: 0.1,_id,text,task_1,task_2,text_clean
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,3,made amp amp onli abl start make money sustain...
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,2,technic still turn back clock dick head
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,1,govt stop think world media liber gang ani opt...
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,2,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,2,would better ask think sleazi shitbag lmao


In [ ]:
LE.fit(df['task_2'])
classes= list(LE.classes_)
classes

[0, 1, 2, 3]

In [ ]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text_clean'], df['task_2'], 
                                                                    random_state=42, 
                                                                    test_size=0.1)

In [ ]:
X_train = train_text.tolist()
X_test = temp_text.tolist()

y_train = train_labels.tolist()
y_test = temp_labels.tolist()

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'  # replace this with model of choice
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=classes)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
history = learner.fit_onecycle(5e-5, 2)
learner.validate(class_names=classes) # class_names must be string values

preprocessing train...
language: en
train sequence lengths:
	mean : 14
	95percentile : 27
	99percentile : 31


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 14
	95percentile : 27
	99percentile : 31




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/2
576/576 [==============================] - 7733s 13s/step - loss: 1.0506 - accuracy: 0.5520 - val_loss: 1.0080 - val_accuracy: 0.5599
Epoch 2/2
233/576 [===========>..................] - ETA: 1:15:02 - loss: 0.8624 - accuracy: 0.6538

In [ ]:
classes

[0, 1, 2, 3]